# Data science Capstone project -- Segmenting and cluster Toronto Neighborhood

## 1. Import Libraries and Read From Wiki Page

In [4]:
import pandas as pd
import numpy as np

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url,header=0)

In [6]:
neighbor_df = tables[0]
neighbor_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2. Data Clean Up and Wrangling

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
neighbor_df.columns=['PostalCode','Borough','Neighborhood']

In [8]:
neighbor_df.shape

(288, 3)

In [9]:
neighbor_df[neighbor_df.Borough=='Not assigned'].shape

(77, 3)

In [10]:
neighbor_df = neighbor_df[neighbor_df.Borough!='Not assigned']

In [11]:
neighbor_df.shape

(211, 3)

More than one neighborhood can exist in one postal code area. For example, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [12]:
neighbor_df[neighbor_df.PostalCode=='M5A']

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [13]:
neighbor = neighbor_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighbor.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [14]:
neighbor.loc[neighbor.Neighborhood=='Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [15]:
neighbor.loc[neighbor.Neighborhood=='Not assigned','Neighborhood'] = neighbor.loc[neighbor.Neighborhood=='','Borough']

In [16]:
neighbor[neighbor.Neighborhood=='Not assigned']

,PostalCode,Borough,Neighborhood


In [17]:
neighbor.shape

(103, 3)

## 3. Load Neiborhood Location Geocode

In [20]:
geocode_df = pd.read_csv('Geospatial_Coordinates.csv')
geocode_df.columns=['PostalCode','Latitude','Longitude']
geocode_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
neighbor_geocode = pd.merge(neighbor, geocode_df, on='PostalCode')
print(neighbor_geocode.shape)
neighbor_geocode.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
